In [18]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from kaggle.api.kaggle_api_extended import KaggleApi
import kagglehub

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms, models
import torch.optim as optim

In [3]:
api = KaggleApi()
api.authenticate()

In [4]:
!chcp 65001
path = %pwd

Active code page: 65001


In [5]:
#api.dataset_download_files('swoyam2609/fresh-and-stale-classification', path = path, unzip = True)

Dataset URL: https://www.kaggle.com/datasets/swoyam2609/fresh-and-stale-classification


In [6]:
os.listdir()

['.ipynb_checkpoints', 'dataset', 'Untitled.ipynb']

In [25]:
os.chdir('dataset/Train')
len(os.listdir())

18

In [12]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  
])

In [14]:
train_dataset = datasets.ImageFolder(root='dataset/Train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = datasets.ImageFolder(root='dataset/Test', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [28]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Первый слой свертки
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Второй слой свертки
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        # Полносвязные слои
        self.fc1 = nn.Linear(56*56*64, 128)  # Учли новые размеры!
        self.fc2 = nn.Linear(128, 18)  # Классификация на 10 классов
    
    def forward(self, x):
        # Первая свертка
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # Вторая свертка
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # Приводим к линейному виду
        x = x.view(-1, 56*56*64)
        
        # Полносвязные слои
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x

In [29]:
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()  # Функция потерь
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Оптимизатор Adam

epochs = 5
for epoch in range(epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Эпоха {epoch+1}, Потеря: {loss.item():.4f}')

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/Train\\rottenbanana\\rotated_by_15_Screen Shot 2018-06-12 at 9.21.25 PM.png'

In [32]:
train_dataset[0].shape

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/Train\\freshapples\\Screen Shot 2018-06-08 at 4.59.36 PM.png'